In [ ]:
from ej.all import *
from boogie.models.wordcloud import word_cloud

In [ ]:
qs = clusterization.clusters.all()
qs

In [ ]:
import numpy as np

In [ ]:
c = clusterizations.first()
qs = c.clusters.all()


# qs.values_list('users', 'id').dataframe()
conversations = qs.values_list('clusterization__conversation').distinct()
votes = (Vote.objects
 .filter(comment__conversation__in=conversations)
 .pivot_table('author', 'comment', 'choice', fill_value=0))

votes['cluster'] = qs.dataframe('id', index='users')
centroids = votes.groupby('cluster').mean()
groups = votes.groupby('cluster')
groups = {k: groups.get_group(k).drop(columns='cluster') for k in groups.groups}

raw = votes.drop(columns='cluster')
x = raw.values[0]
def second_closest(x):
    distances = ((x - centroids)**2).sum(axis=1)
    idx = distances.values.argsort()[1]
    cluster_idx = centroids.index[idx]
    return pd.Series([cluster_idx, distances.loc[cluster_idx]], index=('cluster', 'distance'))
    
distances = votes.drop(columns='cluster').aggregate(second_closest, axis=1)
g = distances.groupby('cluster')
g.agg(['min', 'idxmin'])

In [ ]:
from django.test import RequestFactory

rf = RequestFactory()
r = rf.get('/home/')
r.path

In [ ]:
r = client.request

In [161]:
g.aggregate?

In [3]:
votes = conversation.comments.votes().dataframe('author', 'comment', 'choice')

In [3]:
group = votes.groupby(['comment', 'choice'])
df = group.count()
extra = df.index.to_frame()
df['comment'] = extra['comment']
df['choice'] = extra['choice']
df.index = df.reindex()
table = df.pivot_table(index='comment', columns='choice', values='author', fill_value=0)

In [4]:
total = table.sum(axis=1)
table_pc = pd.DataFrame(100 * table.values / total.values[:, None], columns=table.columns, index=table.index)

In [5]:
table_pc.applymap(lambda x: round(x, 2))

choice,-1,0,1
comment,,,
1,97.44,2.56,0.00
2,36.36,54.55,9.09
3,28.57,71.43,0.00
4,69.23,1.92,28.85
5,54.55,27.27,18.18
6,65.79,5.26,28.95


In [6]:
table_pc.round(1)

choice,-1,0,1
comment,,,
1,97.4,2.6,0.0
2,36.4,54.5,9.1
3,28.6,71.4,0.0
4,69.2,1.9,28.8
5,54.5,27.3,18.2
6,65.8,5.3,28.9


In [7]:
table[-1]

comment
1    38
2    12
3     4
4    36
5    18
6    25
Name: -1, dtype: int64

In [8]:
table[2] = 0

In [9]:
table

choice,-1,0,1,2
comment,,,,
1,38,1,0,0
2,12,18,3,0
3,4,10,0,0
4,36,1,15,0
5,18,9,6,0
6,25,2,11,0


54

In [9]:
import dis

@dis.dis
def foo(x):
    try:
        return cos(x, y, z, h)
    except:
        raise
        return 1

  5           0 SETUP_EXCEPT            14 (to 16)

  6           2 LOAD_GLOBAL              0 (cos)
              4 LOAD_FAST                0 (x)
              6 LOAD_GLOBAL              1 (y)
              8 LOAD_GLOBAL              2 (z)
             10 LOAD_GLOBAL              3 (h)
             12 CALL_FUNCTION            4
             14 RETURN_VALUE

  7     >>   16 POP_TOP
             18 POP_TOP
             20 POP_TOP

  8          22 LOAD_GLOBAL              4 (Foo)
             24 LOAD_CONST               1 (1)
             26 CALL_FUNCTION            1
             28 RAISE_VARARGS            1

  9          30 LOAD_CONST               1 (1)
             32 RETURN_VALUE
             34 LOAD_CONST               0 (None)
             36 RETURN_VALUE


In [13]:
v.aggregate(agree=Count('choice', filter=Q(choice=1)), 
            disagree=Count('choice', filter=Q(choice=-1)), 
            skip=Count('choice', filter=Q(choice=0)), 
            total=Count('choice'), 
           )

{'agree': 35, 'disagree': 133, 'skip': 41, 'total': 209}